In [ ]:
import pandas as pd
import numpy as np
import json
import glob, os, subprocess
import geocoder
import time
import inspect

In [ ]:
logs = pd.read_csv('/home/kin672/gentb-summer22/Geography/gentb.haproxy.log', sep="\t", header=None)
logs.columns = ["original"]

In [ ]:
# Isolate the ip addresses
temp_ip = logs['original'].str.split(':',expand=True)
logs['ip'] = ''
for index, row in temp_ip.iterrows():
    row = np.array(row)
    check = 'messages' in row[0]
    if check:
        ip = row[4]
    else:
        ip = row[3]
    if ip == '':
        print(row)
    ip = ip.replace(' ', '')
    logs.at[index, 'ip'] = ip

In [ ]:
# Isolate the UUIDs
temp_uuid= logs['original'].str.split('/',expand=True)
logs['uuid'] = ''
for index, row in temp_uuid.iterrows():
    row = np.array(row)
    row_len = np.array([len(str(x)) for x in row])
    uuids = list(row[np.where(row_len == 32)])
    if len(uuids) == 1:
        uuid = uuids[0]
        if ' ' in uuid or '=' in uuid or '-' in uuid:
            uuid = 'No UUID'
        logs.at[index, 'uuid'] = uuid
    elif len(uuids) == 0:
        logs.at[index, 'uuid'] = 'No UUID'
    else:
        logs.at[index, 'uuid'] = 'Too many UUIDs?'

In [ ]:
# Download the unique ips as a csv
pd.DataFrame(logs.ip.unique()).to_csv('/home/kin672/gentb-summer22/Geography/unique_ips.csv', index = False)

In [ ]:
# Convert a list of ip addresses to full set of location info [NOT WORKING]
def get_locations(fNames):

    latlng = []
    country = []
    region = []
    city = []
    
    for ip in fNames:
        try:
            proc = subprocess.Popen(f"curl ipinfo.io/{ip}?token=80efdb52ccb9b2", shell=True, encoding='utf8', stdout=subprocess.PIPE)
            output = proc.communicate()[0]

            latlng.append(output.loc)
            country.append(output.country)
            region.append(output.region)
            city.append(output.city)
        except:
            latlng.append('Failed')
            country.append('Failed')
            region.append('Failed')
            city.append('Failed')
        
    return pd.DataFrame({"IP": fNames, "LatLng": latlng, 'Country': country, 'region': region, 'city' : city})


In [ ]:
# Testing function above
fNames = ['114.119.136.111']
get_locations(fNames)

In [ ]:
# Alternative method if you have enough calls to ipinfo (how to use API key?)
ip_guide = {}
for ip in logs.ip.unique():
    temp = {'latlng':[], 'country':[], 'city':[], 'region': []}
    geoitem = geocoder.ip(ip)
    time.sleep(30)
    temp['latlng'] = geoitem.latlng
    temp['country'] = geoitem.country
    temp['city'] = geoitem.city
    temp['region'] = geoitem.region
    ip_guide[ip] = temp